In [2]:
import sys

!{sys.executable} -m pip install folium
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install requests
import numpy as np
import pandas as pd 
import folium 
import requests

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [14]:
def get_and_format_trip_data(file_name):
    '''
    input: file name
    output: pandas data frame
    '''
    bike_trips = pd.read_csv(file_name)

    bike_trips["start_time"] = pd.to_datetime(bike_trips["start_time"])
    bike_trips["end_time"] = pd.to_datetime(bike_trips["end_time"])
    bike_trips["hour"] = bike_trips["start_time"].map(lambda x: x.hour)
    return bike_trips

In [15]:
trips = get_and_format_trip_data('Divvy_Trips_2018_0405.csv')
print(trips.head(10))

         trip_id          start_time            end_time  bikeid  \
0       18709068 2018-05-31 23:59:00 2018-06-01 00:05:00    2233   
1       18709067 2018-05-31 23:59:00 2018-06-01 00:07:00    3904   
2       18709066 2018-05-31 23:58:00 2018-06-01 00:08:00    1969   
3       18709065 2018-05-31 23:58:00 2018-06-01 00:21:00     457   
4       18709064 2018-05-31 23:57:00 2018-06-01 00:08:00    4129   
5       18709063 2018-05-31 23:57:00 2018-06-01 00:08:00    4156   
6       18709062 2018-05-31 23:57:00 2018-06-01 00:06:00    2260   
7       18709061 2018-05-31 23:57:00 2018-06-01 00:17:00    6112   
8       18709060 2018-05-31 23:56:00 2018-06-01 00:02:00    2604   
9       18709059 2018-05-31 23:56:00 2018-06-01 00:08:00    4310   
10      18709058 2018-05-31 23:56:00 2018-06-01 00:15:00      99   
11      18709057 2018-05-31 23:56:00 2018-06-01 00:06:00    2411   
12      18709056 2018-05-31 23:56:00 2018-06-01 00:09:00     681   
13      18709055 2018-05-31 23:56:00 2018-06-01 

In [16]:
print(trips.head(10))

    trip_id          start_time            end_time  bikeid  tripduration  \
0  18709068 2018-05-31 23:59:00 2018-06-01 00:05:00    2233           367   
1  18709067 2018-05-31 23:59:00 2018-06-01 00:07:00    3904           521   
2  18709066 2018-05-31 23:58:00 2018-06-01 00:08:00    1969           563   
3  18709065 2018-05-31 23:58:00 2018-06-01 00:21:00     457          1410   
4  18709064 2018-05-31 23:57:00 2018-06-01 00:08:00    4129           634   
5  18709063 2018-05-31 23:57:00 2018-06-01 00:08:00    4156           644   
6  18709062 2018-05-31 23:57:00 2018-06-01 00:06:00    2260           539   
7  18709061 2018-05-31 23:57:00 2018-06-01 00:17:00    6112          1237   
8  18709060 2018-05-31 23:56:00 2018-06-01 00:02:00    2604           343   
9  18709059 2018-05-31 23:56:00 2018-06-01 00:08:00    4310           686   

   from_station_id           from_station_name  to_station_id  \
0               21  Aberdeen St & Jackson Blvd            198   
1              172    

In [20]:
def get_station_list():
    '''output: pandas data frame'''
    station_api_key = 'stationBeanList'
    try:
        response = requests.get('https://feeds.divvybikes.com/stations/stations.json')
        station_data = response.json()
        station_list = station_data[station_api_key]
        stations = pd.DataFrame.from_records(station_list, index='id')
    except:
        print('something went wrong')
    return stations

In [21]:
stations = get_station_list()

In [22]:
print(stations.head(10))

   altitude  availableBikes  availableDocks     city  is_renting kioskType  \
id                                                                           
2                         7              32  Chicago        True   K3_FULL   
3                         6              47  Chicago        True   K3_FULL   
4                        11              12  Chicago        True   K3_FULL   
5                         6              17  Chicago        True   K3_FULL   
6                        10              23  Chicago        True   K3_FULL   
7                         2              15  Chicago        True   K3_FULL   
9                         7               7  Chicago        True   K3_FULL   
11                        6               4  Chicago        True   K3_FULL   
12                        3              12  Chicago        True   K3_FULL   
13                       22              13  Chicago        True   K3_FULL   

   landMark lastCommunicationTime   latitude           location

In [25]:
def add_trip_counts_to_stations(stations, trips):
    '''
    input: data frames
    output: data frame
    '''
    departure_counts = trips.groupby('from_station_id').count()
    # just for fun, to see the type of data 
    print('trip type', type(trips.groupby('from_station_id')))
    # just for fun, to see how it looks 
    print('what departure counts looks like after gouping and counting', departure_counts.head(10))
    departure_counts = departure_counts.iloc[:, [0]]
    print('type of departure counts after iloc', type(departure_counts))
    print('departure counts after iloc', departure_counts.head(10))
    departure_counts.columns = ['Departure Count']
    print('departure counts after renaming the colun', departure_counts.head(10))

    arrival_counts = trips.groupby('to_station_id').count()
    arrival_counts = arrival_counts.iloc[:, [0]]
    arrival_counts.columns = ['Arrival Count']

    stations = pd.merge(departure_counts, stations,
                        right_on='id',
                        left_index=True).merge(arrival_counts,
                                               left_on='id',
                                               right_index=True)
    return stations

In [26]:
stations = add_trip_counts_to_stations(stations, trips)

trip type <class 'pandas.core.groupby.groupby.DataFrameGroupBy'>
what departure counts looks like after gouping and counting                  trip_id  start_time  end_time  bikeid  tripduration  \
from_station_id                                                        
2                   1914        1914      1914    1914          1914   
3                   4208        4208      4208    4208          4208   
4                   2203        2203      2203    2203          2203   
5                   1345        1345      1345    1345          1345   
6                   2628        2628      2628    2628          2628   
7                   1910        1910      1910    1910          1910   
9                    100         100       100     100           100   
11                    52          52        52      52            52   
12                    80          80        80      80            80   
13                  2072        2072      2072    2072          2072   

          

In [28]:
print(stations.head(10))

    Departure Count altitude  availableBikes  availableDocks     city  \
id                                                                      
2              1914                        7              32  Chicago   
3              4208                        6              47  Chicago   
4              2203                       11              12  Chicago   
5              1345                        6              17  Chicago   
6              2628                       10              23  Chicago   
7              1910                        2              15  Chicago   
9               100                        7               7  Chicago   
11               52                        6               4  Chicago   
12               80                        3              12  Chicago   
13             2072                       22              13  Chicago   

    is_renting kioskType landMark lastCommunicationTime   latitude  \
id                                                   

In [30]:
def put_stations_on_map(stations):
    '''
    input: data frame
    output: folium map
    '''
    map = folium.Map(location=[41.88, -87.62],
                            zoom_start=13,
                            tiles="CartoDB dark_matter")
    for index, station in stations.iterrows():
        popup_text = "{}<br> Total departures: {}<br> Total arrivals: {}<br>"
        popup_text = popup_text.format(stations.at[index, "stationName"],
                                       stations.at[index, "Arrival Count"],
                                       stations.at[index, "Departure Count"],
                                       )
        folium.CircleMarker(location=[stations.at[index, 'latitude'],
                                      stations.at[index, 'longitude']],
                            fill=True,
                            popup=popup_text).add_to(map)
    return map

In [31]:
folium_map = put_stations_on_map(stations)

In [32]:
folium_map